<a href="https://colab.research.google.com/github/Sidhtang/facial-recognition-with-open-cv-and-cnn/blob/main/three_deep_learning_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install ultralytics colorthief deep_sort_realtime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.8/906.8 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 29.3 MB/s eta 0:00:00


In [11]:
# another optimised version of the  code with different prerainrd transformer with temporal smoothing
from ultralytics import YOLO
import cv2
import numpy as np
from collections import Counter
from sklearn.cluster import KMeans
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.models import efficientnet_v2_l, resnet152
from PIL import Image
import csv
from colorthief import ColorThief
import io
import albumentations as A
from deep_sort_realtime.deepsort_tracker import DeepSort
import warnings
warnings.filterwarnings('ignore')

# Set up GPU device with mixed precision training
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device == 'cuda':
    torch.backends.cudnn.benchmark = True
    scaler = torch.cuda.amp.GradScaler()

print(f"Using device: {device}")
class EfficientNetFPN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        # Load EfficientNetV2 as backbone but remove the classifier
        self.backbone = efficientnet_v2_l(pretrained=True)

        # Store the feature extraction points from EfficientNet
        # These correspond to different scales of features
        self.features = {}

        # Remove the classifier and pooling layers as we'll use FPN instead
        self.backbone = nn.Sequential(*list(self.backbone.children())[:-2])

        # Define FPN layers
        # Numbers correspond to EfficientNetV2's channel dimensions
        self.lateral_convs = nn.ModuleList([
            nn.Conv2d(1280, 256, 1),  # For the highest level features
            nn.Conv2d(640, 256, 1),   # For mid-level features
            nn.Conv2d(320, 256, 1)    # For lower-level features
        ])

        # Smooth the merged features
        self.smooth_convs = nn.ModuleList([
            nn.Conv2d(256, 256, 3, padding=1),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.Conv2d(256, 256, 3, padding=1)
        ])

        # Final classifier using multi-scale features
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(256 * 3, 512),  # Concatenate features from all scales
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )

        # Initialize weights properly
        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        # Extract features at different scales from EfficientNet
        features = []
        for i, layer in enumerate(self.backbone):
            x = layer(x)
            if i in {5, 6, 7}:  # Collect features from different scales
                features.append(x)

        # Build top-down pathway (FPN)
        laterals = []
        for feature, lateral_conv in zip(features[::-1], self.lateral_convs):
            # Convert features to common channel dimension
            lat = lateral_conv(feature)
            laterals.append(lat)

        # Process from top to bottom, upsampling and adding features
        fpn_features = [laterals[0]]
        for i in range(len(laterals) - 1):
            # Upsample higher level features
            upsampled = F.interpolate(fpn_features[-1],
                                    size=laterals[i + 1].shape[-2:],
                                    mode='nearest')
            # Add features from lateral connection
            merged = upsampled + laterals[i + 1]
            # Apply smoothing
            smoothed = self.smooth_convs[i](merged)
            fpn_features.append(smoothed)

        # Combine multi-scale features
        multi_scale_features = []
        for fpn_feature in fpn_features:
            pooled = F.adaptive_avg_pool2d(fpn_feature, 1)
            multi_scale_features.append(pooled)

        # Concatenate and classify
        combined = torch.cat(multi_scale_features, dim=1)
        return self.classifier(combined)

class EnhancedColorClassifier(nn.Module):
    def __init__(self):
        super(EnhancedColorClassifier, self).__init__()
        # Using ResNet152 for better color feature extraction
        self.features = resnet152(pretrained=True)
        num_ftrs = self.features.fc.in_features

        # Enhanced classifier head with attention mechanism
        self.attention = nn.Sequential(
            nn.Linear(num_ftrs, 512),
            nn.Tanh(),
            nn.Linear(512, 1)
        )

        self.classifier = nn.Sequential(
            nn.BatchNorm1d(num_ftrs),
            nn.Dropout(0.3),
            nn.Linear(num_ftrs, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.3),
            nn.Linear(512, len(COLOR_CLASSES))
        )

    def forward(self, x):
        with torch.cuda.amp.autocast():
            features = self.features.conv1(x)
            features = self.features.bn1(features)
            features = self.features.relu(features)
            features = self.features.maxpool(features)

            features = self.features.layer1(features)
            features = self.features.layer2(features)
            features = self.features.layer3(features)
            features = self.features.layer4(features)

            features = self.features.avgpool(features)
            features = torch.flatten(features, 1)

            # Apply attention mechanism
            attention_weights = self.attention(features)
            attention_weights = torch.sigmoid(attention_weights)
            attended_features = features * attention_weights

            return self.classifier(attended_features)

# Enhanced constants with more detailed categories
VEHICLE_SUBTYPES = {
    'car': ['sedan', 'suv', 'hatchback', 'wagon', 'coupe', 'sports_car', 'luxury', 'compact',
            'convertible', 'crossover', 'electric_vehicle'],
    'truck': ['pickup', 'semi', 'delivery', 'dump_truck', 'box_truck', 'flatbed', 'tanker',
              'concrete_mixer', 'car_carrier'],
    'bus': ['city_bus', 'coach', 'mini_bus', 'school_bus', 'articulated_bus', 'double_decker',
            'shuttle_bus'],
    'van': ['passenger_van', 'cargo_van', 'minivan', 'camper_van', 'panel_van', 'refrigerated_van',
            'step_van'],
    'two_wheeler': ['motorcycle', 'scooter', 'bicycle', 'electric_bike', 'moped', 'sport_bike',
                   'cruiser', 'touring_bike']
}

COLOR_CLASSES = [
    'black', 'white', 'gray', 'silver', 'red', 'blue', 'green', 'yellow',
    'brown', 'orange', 'purple', 'gold', 'beige', 'burgundy', 'navy',
    'teal', 'bronze', 'copper', 'champagne'
]

def get_enhanced_transforms():
    """
    Create enhanced image preprocessing pipeline with augmentations
    """
    return transforms.Compose([
        transforms.Resize((384, 384)),  # Larger input size for better detail capture
        transforms.RandomHorizontalFlip(p=0.3),
        transforms.RandomAffine(degrees=10, translate=(0.1, 0.1), scale=(0.9, 1.1)),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

def classify_vehicle_details(frame, box, class_name, vehicle_classifier, confidence_threshold=0.7):
    """
    Enhanced vehicle classification with confidence thresholding and error handling
    """
    try:
        x1, y1, x2, y2 = map(int, box)
        vehicle_img = frame[y1:y2, x1:x2]

        # Check if image patch is valid
        if vehicle_img.size == 0 or vehicle_img.shape[0] == 0 or vehicle_img.shape[1] == 0:
            return None

        # Convert to PIL and apply enhanced preprocessing
        img_pil = Image.fromarray(cv2.cvtColor(vehicle_img, cv2.COLOR_BGR2RGB))
        transform = get_enhanced_transforms()
        input_tensor = transform(img_pil).unsqueeze(0).to(device)

        with torch.no_grad():
            with torch.cuda.amp.autocast():
                outputs = vehicle_classifier(input_tensor)
                probabilities = torch.softmax(outputs, dim=1)
                conf, predicted = probabilities.max(1)

                # Only classify if confidence exceeds threshold
                if conf.item() < confidence_threshold:
                    return None

        subtype = list(VEHICLE_SUBTYPES[class_name])[predicted.item() % len(VEHICLE_SUBTYPES[class_name])]

        # Enhanced size classification using relative dimensions
        area = (x2 - x1) * (y2 - y1)
        frame_area = frame.shape[0] * frame.shape[1]
        area_ratio = area / frame_area

        size = 'small' if area_ratio < 0.05 else \
               'medium' if area_ratio < 0.15 else 'large'

        # Enhanced body style and purpose classification
        body_style = 'commercial' if subtype in ['semi', 'delivery', 'dump_truck', 'box_truck', 'tanker',
                                               'concrete_mixer', 'car_carrier', 'refrigerated_van'] else \
                    'passenger' if subtype in ['city_bus', 'school_bus', 'coach', 'shuttle_bus'] else 'standard'

        purpose = 'commercial' if body_style == 'commercial' else \
                 'public' if body_style == 'passenger' else 'personal'

        return {
            'subtype': subtype,
            'size': size,
            'body_style': body_style,
            'purpose': purpose,
            'confidence': conf.item()
        }
    except Exception as e:
        print(f"Vehicle classification error: {e}")
        return None

def setup_models():
    """
    Initialize enhanced models with optimized parameters
    """
    # Initialize YOLOv8x with enhanced parameters
    yolo_model = YOLO('yolov8x.pt')
    yolo_model.conf = 0.35  # Lower confidence threshold for higher recall
    yolo_model.iou = 0.65   # Higher IoU threshold for better precision
    yolo_model.to(device)

    # Initialize enhanced classifiers
    vehicle_classifier = EnhancedVehicleClassifier().to(device)
    vehicle_classifier.eval()

    color_classifier = EnhancedColorClassifier().to(device)
    color_classifier.eval()

    # Initialize DeepSort with optimized parameters
    tracker = DeepSort(
        max_age=45,            # Increased to handle longer occlusions
        n_init=4,              # More frames required for track initialization
        nms_max_overlap=0.85,  # Higher NMS threshold for better tracking
        max_cosine_distance=0.25,  # Stricter matching threshold
        nn_budget=150,         # Larger budget for better tracking history
    )

    return {
        'yolo': yolo_model,
        'vehicle_classifier': vehicle_classifier,
        'color_classifier': color_classifier,
        'tracker': tracker,
        'device': device
    }

def detect_color(frame, box, color_classifier, confidence_threshold=0.6):
    """
    Enhanced color detection with multi-method approach and confidence thresholding
    """
    try:
        x1, y1, x2, y2 = map(int, box)
        vehicle_img = frame[y1:y2, x1:x2]

        if vehicle_img.size == 0:
            return None

        # Convert to PIL Image
        img_pil = Image.fromarray(cv2.cvtColor(vehicle_img, cv2.COLOR_BGR2RGB))

        # Enhanced color detection using multiple methods
        results = {'colors': [], 'confidences': []}

        # Method 1: ML Classification
        transform = get_enhanced_transforms()
        input_tensor = transform(img_pil).unsqueeze(0).to(device)

        with torch.no_grad():
            with torch.cuda.amp.autocast():
                outputs = color_classifier(input_tensor)
                probabilities = torch.softmax(outputs, dim=1)
                conf, predicted = probabilities.max(1)

                if conf.item() >= confidence_threshold:
                    results['colors'].append(COLOR_CLASSES[predicted.item()])
                    results['confidences'].append(conf.item())

        # Method 2: ColorThief analysis
        img_byte_arr = io.BytesIO()
        img_pil.save(img_byte_arr, format='PNG')
        img_byte_arr.seek(0)

        color_thief = ColorThief(img_byte_arr)
        dominant_color = color_thief.get_color(quality=1)
        palette = color_thief.get_palette(color_count=3, quality=1)

        # Convert RGB values to color names using nearest neighbor
        for rgb in [dominant_color] + palette:
            color_name = get_closest_color(rgb)
            if color_name not in results['colors']:
                results['colors'].append(color_name)
                results['confidences'].append(0.8)  # Default confidence for color matching

        # Return most confident color
        if results['colors']:
            max_conf_idx = np.argmax(results['confidences'])
            return {
                'color': results['colors'][max_conf_idx],
                'confidence': results['confidences'][max_conf_idx],
                'secondary_colors': results['colors'][:3]  # Top 3 detected colors
            }
        return None

    except Exception as e:
        print(f"Color detection error: {e}")
        return None

def get_closest_color(rgb):
    """
    Convert RGB value to the closest named color using an extended color mapping.
    """
    color_map = {
        'black': (0, 0, 0),
        'white': (255, 255, 255),
        'gray': (128, 128, 128),
        'silver': (192, 192, 192),
        'red': (255, 0, 0),
        'blue': (0, 0, 255),
        'green': (0, 255, 0),
        'yellow': (255, 255, 0),
        'brown': (165, 42, 42),
        'orange': (255, 165, 0),
        'purple': (128, 0, 128),
        'gold': (255, 215, 0),
        'beige': (245, 245, 220),
        'burgundy': (128, 0, 32),
        'navy': (0, 0, 128),
        'teal': (0, 128, 128),
        'bronze': (205, 127, 50),
        'copper': (184, 115, 51),
        'champagne': (247, 231, 206),
        'pink': (255, 192, 203),
        'cyan': (0, 255, 255),
        'lime': (191, 255, 0),
        'magenta': (255, 0, 255),
        'olive': (128, 128, 0),
        'peach': (255, 229, 180),
    }

    min_distance = float('inf')
    closest_color = None

    for color_name, color_rgb in color_map.items():
        distance = sum((a - b) ** 2 for a, b in zip(rgb, color_rgb))
        if distance < min_distance:
            min_distance = distance
            closest_color = color_name

    return closest_color


    for color_name, color_rgb in color_map.items():
        distance = sum((a - b) ** 2 for a, b in zip(rgb, color_rgb))
        if distance < min_distance:
            min_distance = distance
            closest_color = color_name

    return closest_color

def process_video(video_path, output_path, models, data_output_path):
    """
    Enhanced video processing with multi-frame analysis and temporal smoothing
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError(f"Could not open video: {video_path}")

    # Video writer setup
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

    data = []
    frame_count = 0
    temporal_buffer = {}  # Store recent detections for temporal smoothing

    try:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            frame_count += 1

            # Process every frame for maximum accuracy
            detection_list = []

            # Run YOLO detection with test-time augmentation
            results = models['yolo'](frame, augment=True)  # Enable test-time augmentation

            # Process YOLO detections
            for box, conf, cls in zip(results[0].boxes.xyxy, results[0].boxes.conf, results[0].boxes.cls):
                x1, y1, x2, y2 = map(int, box.tolist())
                detection = ([x1, y1, x2 - x1, y2 - y1], conf.item(), int(cls.item()))
                detection_list.append(detection)

            # Update tracks with enhanced DeepSort
            tracks = models['tracker'].update_tracks(detection_list, frame=frame)

            # Process each track with temporal smoothing
            for track in tracks:
                if not track.is_confirmed():
                    continue

                track_id = track.track_id
                ltwh = track
                to_ltwh = track.to_ltwh()
                box = [int(to_ltwh[0]), int(to_ltwh[1]),
                      int(to_ltwh[0] + to_ltwh[2]), int(to_ltwh[1] + to_ltwh[3])]

                cls_id = track.get_det_class()
                if cls_id is None:
                    continue

                class_name = models['yolo'].names[cls_id]

                if class_name in VEHICLE_SUBTYPES:
                    # Initialize temporal buffer for this track if needed
                    if track_id not in temporal_buffer:
                        temporal_buffer[track_id] = {
                            'vehicle_details': [],
                            'color_info': [],
                            'frame_history': []
                        }

                    # Get vehicle details
                    vehicle_details = classify_vehicle_details(
                        frame, box, class_name,
                        models['vehicle_classifier']
                    )

                    # Get color information
                    color_info = detect_color(
                        frame, box,
                        models['color_classifier']
                    )

                    # Update temporal buffer
                    if vehicle_details and color_info:
                        temporal_buffer[track_id]['vehicle_details'].append(vehicle_details)
                        temporal_buffer[track_id]['color_info'].append(color_info)
                        temporal_buffer[track_id]['frame_history'].append(frame_count)

                        # Keep only recent history (last 5 frames)
                        max_history = 5
                        if len(temporal_buffer[track_id]['vehicle_details']) > max_history:
                            temporal_buffer[track_id]['vehicle_details'] = temporal_buffer[track_id]['vehicle_details'][-max_history:]
                            temporal_buffer[track_id]['color_info'] = temporal_buffer[track_id]['color_info'][-max_history:]
                            temporal_buffer[track_id]['frame_history'] = temporal_buffer[track_id]['frame_history'][-max_history:]

                        # Get smoothed predictions using temporal buffer
                        smoothed_details = get_smoothed_predictions(temporal_buffer[track_id])

                        # Create detailed label with confidence scores
                        label_parts = [
                            f"{smoothed_details['subtype']} ({smoothed_details['size']})",
                            f"#{track_id}",
                            f"{smoothed_details['color']}",
                            f"{smoothed_details['confidence']:.2f}"
                        ]
                        label = " ".join(label_parts)

                        # Calculate dynamic color based on confidence
                        confidence = smoothed_details['confidence']
                        bbox_color = (
                            int(255 * (1 - confidence)),  # Red component
                            int(255 * confidence),        # Green component
                            0                            # Blue component
                        )

                        # Enhanced visualization
                        cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), bbox_color, 2)

                        # Add background to text for better visibility
                        text_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)[0]
                        cv2.rectangle(frame,
                                    (box[0], box[1] - 25),
                                    (box[0] + text_size[0], box[1]),
                                    bbox_color, -1)
                        cv2.putText(frame, label, (box[0], box[1] - 10),
                                  cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

                        # Store detection data with enhanced information
                        data.append({
                            'Frame': frame_count,
                            'Track_ID': track_id,
                            'Main_Type': class_name,
                            'Subtype': smoothed_details['subtype'],
                            'Size': smoothed_details['size'],
                            'Body_Style': smoothed_details['body_style'],
                            'Purpose': smoothed_details['purpose'],
                            'Primary_Color': smoothed_details['color'],
                            'Secondary_Colors': smoothed_details['secondary_colors'],
                            'Detection_Confidence': track.get_det_conf() or 0.0,
                            'Classification_Confidence': smoothed_details['confidence'],
                            'Box_Coordinates': box,
                            'Temporal_Confidence': len(temporal_buffer[track_id]['vehicle_details']) / max_history
                        })

            # Clean up old tracks from temporal buffer
            current_track_ids = {track.track_id for track in tracks if track.is_confirmed()}
            temporal_buffer = {k: v for k, v in temporal_buffer.items() if k in current_track_ids}

            # Add frame counter and processing statistics
            cv2.putText(frame, f"Frame: {frame_count}", (10, 30),
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            out.write(frame)

    finally:
        # Release resources
        cap.release()
        out.release()

        # Save detection data with enhanced error handling
        if data:
            try:
                with open(data_output_path, 'w', newline='') as f:
                    writer = csv.DictWriter(f, fieldnames=data[0].keys())
                    writer.writeheader()
                    writer.writerows(data)
                print(f"Successfully saved detection data to {data_output_path}")
            except Exception as e:
                print(f"Error saving detection data: {e}")
                # Attempt to save to alternative location
                backup_path = "backup_" + data_output_path
                with open(backup_path, 'w', newline='') as f:
                    writer = csv.DictWriter(f, fieldnames=data[0].keys())
                    writer.writeheader()
                    writer.writerows(data)
                print(f"Saved backup detection data to {backup_path}")

def get_smoothed_predictions(track_buffer):
    """
    Calculate smoothed predictions using temporal buffer data
    """
    # Get recent vehicle details
    recent_details = track_buffer['vehicle_details']
    recent_colors = track_buffer['color_info']

    if not recent_details or not recent_colors:
        return None

    # Count occurrences of each prediction
    subtype_counts = Counter(d['subtype'] for d in recent_details)
    size_counts = Counter(d['size'] for d in recent_details)
    body_style_counts = Counter(d['body_style'] for d in recent_details)
    purpose_counts = Counter(d['purpose'] for d in recent_details)
    color_counts = Counter(c['color'] for c in recent_colors)

    # Calculate average confidence
    avg_confidence = np.mean([d['confidence'] for d in recent_details])

    # Get secondary colors from recent detections
    all_secondary_colors = []
    for color_info in recent_colors:
        if 'secondary_colors' in color_info:
            all_secondary_colors.extend(color_info['secondary_colors'])
    secondary_colors = [color for color, count in Counter(all_secondary_colors).most_common(3)]

    # Return smoothed predictions
    return {
        'subtype': subtype_counts.most_common(1)[0][0],
        'size': size_counts.most_common(1)[0][0],
        'body_style': body_style_counts.most_common(1)[0][0],
        'purpose': purpose_counts.most_common(1)[0][0],
        'color': color_counts.most_common(1)[0][0],
        'secondary_colors': secondary_colors,
        'confidence': avg_confidence
    }

def main():
    """
    Main function with enhanced error handling and logging
    """
    try:
        # Define paths
        video_path = "/content/trimmed version of the input video.mp4"
        output_path = "output_video.mp4"
        data_output_path = "vehicle_tracking_data.csv"

        print("Initializing models...")
        models = setup_models()
        print("Models initialized successfully")

        print("Starting video processing...")
        process_video(video_path, output_path, models, data_output_path)
        print("Video processing completed successfully")

    except Exception as e:
        print(f"Error in main execution: {e}")
        raise

if __name__ == "__main__":
    main()

Using device: cpu
Initializing models...


Downloading: "https://download.pytorch.org/models/efficientnet_v2_l-59c71312.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_v2_l-59c71312.pth
100%|██████████| 455M/455M [00:08<00:00, 54.6MB/s]
Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to /root/.cache/torch/hub/checkpoints/resnet152-394f9c45.pth
100%|██████████| 230M/230M [00:02<00:00, 107MB/s]


Models initialized successfully
Starting video processing...
Error in main execution: Could not open video: /content/trimmed version of the input video.mp4


ValueError: Could not open video: /content/trimmed version of the input video.mp4